In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
itPostsRows = sc.textFile("../Data/Italian_Stack_Exchange/italianPosts.csv")
itPostsSplit = itPostsRows.map(lambda x: x.split("~"))
from pyspark.sql import Row
from datetime import datetime
def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None
    
def stringToPost(row):
  r = row.encode('utf8').split("~")
  return Row(
    toIntSafe(r[0]),
    toTimeSafe(r[1]),
    toIntSafe(r[2]),
    r[3],
    toIntSafe(r[4]),
    toTimeSafe(r[5]),
    toIntSafe(r[6]),
    toIntSafe(r[7]),
    r[8],
    toIntSafe(r[9]),
    toLongSafe(r[10]),
    toLongSafe(r[11]),
    long(r[12]))
from pyspark.sql.types import *
postSchema = StructType([
  StructField("commentCount", IntegerType(), True),
  StructField("lastActivityDate", TimestampType(), True),
  StructField("ownerUserId", LongType(), True),
  StructField("body", StringType(), True),
  StructField("score", IntegerType(), True),
  StructField("creationDate", TimestampType(), True),
  StructField("viewCount", IntegerType(), True),
  StructField("title", StringType(), True),
  StructField("tags", StringType(), True),
  StructField("answerCount", IntegerType(), True),
  StructField("acceptedAnswerId", LongType(), True),
  StructField("postTypeId", LongType(), True),
  StructField("id", LongType(), False)
  ])
rowRDD = itPostsRows.map(lambda x: stringToPost(x))
itPostsDFStruct = sqlContext.createDataFrame(rowRDD, postSchema)

In [21]:
itPostsDFIdBody = itPostsDFStruct.select("id", "body")
itPostsDFIdBody.show()

+----+--------------------+
|  id|                body|
+----+--------------------+
|1165|&lt;p&gt;The infi...|
|1166|&lt;p&gt;Come cre...|
|1167|&lt;p&gt;Il verbo...|
|1168|&lt;p&gt;As part ...|
|1169|&lt;p&gt;&lt;em&g...|
|1170|&lt;p&gt;There's ...|
|1171|&lt;p&gt;As other...|
|1172|&lt;p&gt;The expr...|
|1173|&lt;p&gt;When I w...|
|1174|&lt;p&gt;Wow, wha...|
|1175|&lt;p&gt;Suppose ...|
|1176|&lt;p&gt;Except w...|
|1177|&lt;p&gt;Both you...|
|1178|&lt;blockquote&gt...|
|1179|&lt;p&gt;Comparin...|
|1180|&lt;p&gt;Using th...|
|1181|&lt;p&gt;I would ...|
|1182|&lt;p&gt;Putting ...|
|1183|&lt;p&gt;Many peo...|
|1184|&lt;p&gt;Sono un'...|
+----+--------------------+
only showing top 20 rows



In [22]:
itPostsDFIdBody = itPostsDFStruct.select(itPostsDFStruct["id"], itPostsDFStruct["body"])
itPostsDFIdBody.show()

+----+--------------------+
|  id|                body|
+----+--------------------+
|1165|&lt;p&gt;The infi...|
|1166|&lt;p&gt;Come cre...|
|1167|&lt;p&gt;Il verbo...|
|1168|&lt;p&gt;As part ...|
|1169|&lt;p&gt;&lt;em&g...|
|1170|&lt;p&gt;There's ...|
|1171|&lt;p&gt;As other...|
|1172|&lt;p&gt;The expr...|
|1173|&lt;p&gt;When I w...|
|1174|&lt;p&gt;Wow, wha...|
|1175|&lt;p&gt;Suppose ...|
|1176|&lt;p&gt;Except w...|
|1177|&lt;p&gt;Both you...|
|1178|&lt;blockquote&gt...|
|1179|&lt;p&gt;Comparin...|
|1180|&lt;p&gt;Using th...|
|1181|&lt;p&gt;I would ...|
|1182|&lt;p&gt;Putting ...|
|1183|&lt;p&gt;Many peo...|
|1184|&lt;p&gt;Sono un'...|
+----+--------------------+
only showing top 20 rows



In [23]:
itPostsDFStruct.drop("body").show()

+------------+--------------------+-----------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|score|        creationDate|viewCount|title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+
|           4|2013-11-11 18:21:...|         17|   23|2013-11-10 19:37:...|     null| null|                    |       null|            null|         2|1165|
|           5|2013-11-10 20:31:...|         12|    1|2013-11-10 19:44:...|       61| null| &lt;word-choice&gt;|          1|            null|         1|1166|
|           2|2013-11-10 20:31:...|         17|    5|2013-11-10 19:58:...|     null| null|                    |       null|            null|         2|1167|
|           1|2014-07-25 13:15:...|        154|   11|2013-

In [33]:
itPostsDFFilteredId = itPostsDFStruct.where("id > 2000 and id < 2010")
#Same as..
#itPostsDFFilteredId = itPostsDFStruct.filter("id > 2000 and id < 2010")

In [31]:
itPostsDFFilteredId.select("id","body").show()

+----+--------------------+
|  id|                body|
+----+--------------------+
|2001|&lt;p&gt;Sardinia...|
|2002|&lt;p&gt;I am fro...|
|2003|&lt;p&gt;La rispo...|
|2004|&lt;p&gt;In itali...|
|2005|&lt;p&gt;Mi Ã¨ st...|
|2006|&lt;p&gt;âCa.â...|
|2007|&lt;p&gt;Un chimi...|
|2008|&lt;p&gt;&quot;sp...|
|2009|&lt;p&gt;Ad occhi...|
+----+--------------------+



In [26]:
itPostsDFFilteredId.withColumnRenamed("id","selected_id")

DataFrame[commentCount: int, lastActivityDate: timestamp, ownerUserId: bigint, body: string, score: int, creationDate: timestamp, viewCount: int, title: string, tags: string, answerCount: int, acceptedAnswerId: bigint, postTypeId: bigint, selected_id: bigint]

In [27]:
itPostsDFWithRatio = itPostsDFFilteredId.withColumn("score_div_answer",itPostsDFFilteredId['score']/itPostsDFFilteredId['answerCount'])

In [28]:
itPostsDFWithRatio.where(itPostsDFWithRatio['score_div_answer'].isNotNull()).show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+----------------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|score_div_answer|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+----------------+
|           8|2014-06-14 07:37:...|        223|&lt;p&gt;Mi Ã¨ st...|    2|2014-06-05 12:04:...|      104| null|&lt;word-usage&gt...|          1|            2006|         1|2005|             2.0|
|           1|2014-07-02 19:39:...|        223|&lt;p&gt;Un chimi...|    2|2014-06-06 11:34:...|       45| null|&lt;word-usage&gt...|          1|            null|         1|2007|             2.0|
+------------+-----------

In [37]:
itPostsDFWithRatio.sort("id", ascending=0).show()
# same as ..
#itPostsDFWithRatio.orderBy("id", ascending=0).show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+----------------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|score_div_answer|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+--------------------+-----------+----------------+----------+----+----------------+
|           0|2014-06-06 13:18:...|        674|&lt;p&gt;Ad occhi...|    5|2014-06-06 13:18:...|     null| null|                    |       null|            null|         2|2009|            null|
|           3|2014-06-07 12:11:...|        676|&lt;p&gt;&quot;sp...|    0|2014-06-06 12:29:...|     null| null|                    |       null|            null|         2|2008|            null|
|           1|2014-07-02 